## Обучение генеративной трансформерной модели с помощью `transformers`

В этой работе мы познакомимся на практике с процессом тренировки большой трансформерной языковой модели. Поскольку такая тренировка требует существенных вычислительных ресурсов, выполнять эту работу рекомендуется в Yandex DataSphere, в которой доступны вычислитльные узлы с одни или двумя графическими процессорами Tesla V100.

### Архитектура трансформеров

В рамках этой работы мы предполагаем, что вы уже знакомы с архитектурой трансформеров, например, по [статье из ML-хэндбука](https://academy.yandex.ru/handbook/ml/article/transformery). Также для первоначального знакомства рекомендую заметку [Jay Alammar. The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/), и её частичный [русскоязычный перевод](https://habr.com/ru/articles/486358/).

Мы не будем в рамках работы создавать архитетуру нейросети "с нуля". Если вам инетересно изучить реализацию трансформеров - рекомендую посмотреть на [NanoGPT](https://github.com/karpathy/nanoGPT). Подробно эта реализация разбирается в [этом видео](https://www.youtube.com/watch?v=kCc8FmEb1nY).

### Библиотека `transformers` и её друзья

Стандартом де факто в реализации трансформеров служит библиотека `transformers` от [HuggingFace](http://huggingface.co). Она содержит в себе реализацию большого количества используемых трансформерных архитектур, а также ряд полезных инструментов для их обучения. Многие инструменты также оформлены в виде отдельных библиотек, которые хорошо работают вместе:

* `tokenizers` - быстрая реализация различных токенизаторов, позволяющих разделять входной текст на токены
* `datasets` - манипулирование большими датасетами
* `evaluate` - вычисление различных метрик и оценка результатов обучения
* `accelerate` - реализация вычислений на множестве GPU и на вычислительных кластерах

Для начала, установим необходимые библиотеки:

In [ ]:
%pip install transformers tokenizers datasets evaluate accelerate

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2024.12.0 requires fsspec==2024.12.0.*, but you have fsspec 2024.9.0 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


В текущем варианте при работе в DataSphere возникают проблемы при использовании файлового хранилища. Для решения проблем нам нужно установить последнюю версию библиотеки `s3fs`.

In [ ]:
%pip install --upgrade git+https://github.com/dask/s3fs

  Cloning https://github.com/dask/s3fs to /tmp/pip-req-build-4r4xrjop
  Running command git clone --filter=blob:none --quiet https://github.com/dask/s3fs /tmp/pip-req-build-4r4xrjop
  Resolved https://github.com/dask/s3fs to commit 51e3c80ef380a82081a171de652e2b699753be2b
  Preparing metadata (setup.py) ... done
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.


### Подготовка датасета

В нашем примере, мы будем обучать виртуального Льва Толстого. Для этого, возьмём все основные романы писателя, и подготовим их них датасет. В качестве отправной точки будет использовать тексты из [библиотеки Мошкова](http://lib.ru). Соберем ссылки на романы Анна Каренина, Война и Мир и др. в один список:

In [ ]:
urls = [
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0039.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0040.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0050.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0060.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0070.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0080.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0090.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_1860_dekabristy.shtml",
]

Теперь скачаем все материалы и подготовим из них один большой текстовый файл. Для того нам понадобится убрать HTML-теги, а также несколько первоначальных строчек в каждом из файлов.

In [ ]:
import html
import re

import requests


def download(url):
    return requests.get(url).text


# code borrowed from here: https://github.com/pallets/markupsafe/blob/0.23/markupsafe/__init__.py#L21
striptags_re = re.compile(r"(<!--.*?-->|<[^>]*>)")
entity_re = re.compile(r"&([^;]+);")


def to_text(s):
    return html.unescape(striptags_re.sub("", s))


def beautify(s):
    lines = [x.strip() for x in s.split("\n") if x.strip() != ""]
    for i in range(min(100, len(lines))):
        if lines[i] == "-->":
            break
    return "\n".join(lines[i + 1 :] if i < 100 else lines)


with open("dataset.txt", "w", encoding="utf-8") as f:
    for u in urls:
        text = beautify(to_text(download(u)))
        f.write(text + "\n\n")

In [ ]:
with open("dataset.txt", encoding="utf-8") as f:
      text = f.read()
      print(text[:100])

Лев Николаевич Толстой
Семейное счастье
Оригинал текста: в электронной библиотеке Олега Колесникова



В результате мы получили один большой файл `dataset.txt`, содержащий большой корпус текстов Льва Толстого.

### Датасеты в Yandex DataSphere

При использовании Yandex DataSphere, у нас ограничен объем данных, которые мы можем хранить вместе с проектом. Обычно, большие объемы данных в облаке хранят в **объектном хранилище S3**. DataSphere позволяет легко подключаться к таким хранилищам, монтируя их как обычную директорию в проекте, после чего можно получить доступ к данным как к обычным файлам.

Однако, доступ в хранилище S3 не слишком быстрый, а для обучения сетей хочется отдавать данные как можно быстрее, не тормозя вычислительный процесс. Для этого в DataSphere предусмотрены **датасеты** - это отдельные виртуальные накопители, которые можно легко подключать к различным вычислительным ресурсам.

Будучи созданным, датасет не может быть изменён - это обеспечивает сохранность исходных данных. Хорошим стилем считается хранить все данные для обучения моделей в датасетах. Кроме того, датасеты можно разделять между другими участниками сообщества или проекта.

В нашем случае объем обучающих данных небольшой, и можно обойтись без создания датасета. Но если вы хотите попробовать - добавьте ниже ячейку со следующим кодом и запустите его:
```
#!:bash
#pragma dataset init mytext --size 1Gb
cp dataset.txt /home/jupyter/mnt/datasets/mytext
```
Это создаст датасет `mytext` с единственным файлом `dataset.txt`. При этом ниже в коде вам нужно будет изменить путь к файлу `dataset.txt` на `/home/jupyter/mnt/datasets/mytext/dataset.txt`.

> Кажется, что в создании датасета нет большого смысла, поскольку мы просто положили тот же файл в другое место. На самом деле это не так - теперь файл `dataset.txt` не будет занимать место в хранилище проекта, доступ к нему будет быстрее, а также вы сможете легко поделиться датасетом с другими участниками команды, чтобы им не пришлось писать код по предварительной обработке данных. При этом датасет не будет копироваться, а будет просто смонтирован в соответствующие директории в DataSphere.

### Токенизация

Нейросети работают с числами, поэтому первым этапом является токенизация текста, т.е. разбиение его на атомарные элементы, которые затем можно добавить в словарь, и представлять текст как последовательность индексов в словаре. Текст можно токенизировать по буквам, или по словам.

При построении современных генеративных сетей текст обычно разбивают на фрагменты таким образом, чтобы частота появления каждого фрагмента в тексте была примерно одинакова. Это лежит в основе т.н. Byte-Pair Encoding (BPE). Подробнее можно прочитать [в этой статье](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt).

Для обучения своего токенизатора используем библиотеку `tokenizers`:

In [1]:
import tokenizers as tok
import transformers as tr

In [ ]:
tokenizer = tok.Tokenizer(tok.models.BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = tok.pre_tokenizers.Whitespace()
trainer = tok.trainers.BpeTrainer(special_tokens=["[PAD]"])
tokenizer.train(["dataset.txt"], trainer)
tokenizer.enable_padding()

А данном случае мы используем два специальных токена - `[UNK]` для представления неизвестного токена (такое случится, если на вход попадёт символ, который токенизатор не видел при обучении), и `[PAD]` для **паддинга** - он используется, если нужно дополнить последовательность до определённой длины.

Вот как можно закодировать входной текст:

In [ ]:
tokenizer.encode("Иван Сигизмундович подошел к окну и закашлялся. Вечерело.").tokens

['Иван',
 'С',
 'иг',
 'изму',
 'н',
 'до',
 'вич',
 'подошел',
 'к',
 'окну',
 'и',
 'закашлялся',
 '.',
 'Вечер',
 'ело',
 '.']

Видим, что популярные слова токенизируются целиком, а те, которые встречаются в тексте редко или не встречаются вовсе - разбиваются на фрагменты.

### Генеративные трансформеры

Для генерации текста используются архитектуры GPT - Generative Pre-trained Transformers. В то время как полноценные трансформеры являются энкодер-декодерной архитектурой, т.е. могут решать задачи преобразования одного вида последовательности в другую, GPT является только декодером, т.к. способно прогнозировать распределение вероятности следующего слова по начальной части последовательности.

Мы используем архитектуру GPT-2, которая, с одной стороны, не слишком огромна, а с другой - может неплохо обучиться. Сперва попробуем натренировать такую архитетуру "с нуля".

Дла начала нам потребуется преобразовать наш токенизатор к объекту `ttokenizer`, который понимает библиотека transformers.

In [ ]:
vocab = tokenizer.get_vocab()
ttokenizer = tr.PreTrainedTokenizerFast(tokenizer_object=tokenizer)
len(vocab)

30000

Теперь создадим непосредственно нейросетевую модель GPT2. При этом основные параметры (количество слоёв, количество голов внимания и т.д. оставим по умолчанию.

In [ ]:
config = tr.GPT2Config(
    vocab_size=len(vocab),
    bos_token_id=tokenizer.token_to_id("[CLS]"),
    eos_token_id=tokenizer.token_to_id("[EOS]"),
)
gpt = tr.GPT2LMHeadModel(config)

Веса вновь созданной модели инициализируются случайным образом, поэтому если мы попросим такую модель сгенерировать текст - получится бессмыслица:

In [ ]:
res = gpt.generate(
    **ttokenizer("Мне нравится ", return_tensors="pt"),
    max_new_tokens=50,
    top_k=3,
    do_sample=True
)
ttokenizer.decode(res[0])

'Мне нравится куртке куртке гениев мальчику усмотре Ката Ката лошаден лошаден измениться жестве убеждены честолюби оголи оголи комитет добрее добрее добрее добрее добрее добрее добрее кобе кобе женный женный женный боитесь спорили grand grand раздви вынесла вынесла раздви раздви доске доске suis поднимался поднимался Вто гусару ухаживать круглыми рик рик брил Comme'

Теперь нам надо научиться подавать на вход модели фрагменты текста для обучения. Для этого существует библиотека `datasets`, входящее в семейство трансформерных библиотек HuggingFace. Помимо того, что эта библиотека умеет работать с разными форматами входных датасетов, она также интегрирована с HuggingFace Hub, и может в одну строчку загружать множество имеющихся на этом сайте датасетов.

В нашем случае мы загрузим датасет из текстового файла:

In [ ]:
import datasets

dataset = datasets.load_dataset("text", data_files="dataset.txt")
dataset["train"][13]

Generating train split: 0 examples [00:00, ? examples/s]

{'text': 'Он взял своею большою рукой меня за руку, и пожал так крепко, честно, только что не больно. Я думала, что он поцелует мою руку, и нагнулась было к нему, но он еще раз пожал мне руку и прямо в глаза посмотрел своим твердым и веселым взглядом.'}

Далее нам необходимо научиться токенизировать датасет, т.е. преобразовывать в числовые тензоры, которые затем мы будем подавать на вход нейросети в процессе обучения. Для этого опишем фукнцию `tokenize`, которая будет возвращать словарь с несколькими полями:

* `input_ids` - это собственно номера слов входной последовательности в словаре
* `token_type_ids` - содержит нули. Это поле используется в более сложных сценариях, например, когда мы тренируем сеть отвечать на вопросы по тексту. В этом случае нам нужно подать на вход текст + вопрос, и это поле позволяет различать между несколькими разными по смыслу фрагментами входной последовательности
* `atttention_mask` показывает, какая часть входной последовательности значима. Для организации последовательности в minibatch нам может потребоваться дополнить последовательность до максимальной длины, и поле `attention_mask` содержит 1 в тех позициях, которые соответствуют исходной последовательности

Такой формат входных данных типичен для трансформерной архитектуры. Также мы передаем последовательность значений целевой переменной `labels`, но поскольку наша задача - это генерация текста, то в качестве `labels` мы передаём копию исходного текста.

In [ ]:
def tokenize(x):
    x = ttokenizer(x["text"])
    x["labels"] = x["input_ids"].copy()
    return x


ds = dataset.map(tokenize, batched=True, remove_columns=["text"])
ds["train"][0]

Map:   0%|          | 0/60781 [00:00<?, ? examples/s]

{'input_ids': [9585, 9735, 3192],
 'token_type_ids': [0, 0, 0],
 'attention_mask': [1, 1, 1],
 'labels': [9585, 9735, 3192]}

Для обучения лучше всего использовать длинные фрагменты текста, поэтому мы сгруппируем все последовательности токенов в блоки размером `block_size`. Для этого мы сначала сконкатенируем все последовательности, а потом разобъем их на блоки. В данном случае мы не будем даже разбивать последовательность на слова и/или предложения - как показывает практика, такой упрощенный подход также даёт хорошие результаты.


In [ ]:
from itertools import chain

block_size = 512 # 1024

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

dsb = ds.map(group_texts, batched=True)

Map:   0%|          | 0/60781 [00:00<?, ? examples/s]

Теперь мы готовы к обучению! Для задания параметров обучения мы создаём объект `TrainingArguments`, в котором задаем директорию, куда будут записываться промежуточные результаты обучения, число эпох, скорость обучения и т.д. Затем на основе этих параметров создаём объект `Trainer`.

Обратите внимание, что размер записываемой на диск сети GPT-2 может быть весьма большим (около 1.4 Gb), что может привести к исчерпанию размера вашей домашней директории в DataSphere. Исходя из этого лучше выбирать параметры `save_steps` и `num_train_epochs` таким образом, чтобы количество записываемых на диск чекпоинтов не превышало 3-5 шт.

Для начала стоит попробовать пообучать сеть в течение 30-90 минут, чтобы увидеть, что она начинает складывать слова более менее правдоподобно.

In [ ]:
targs = tr.TrainingArguments(
    output_dir="gpt2-scratch",
    num_train_epochs=30,
    learning_rate=5e-5,
    warmup_steps=200,
    save_steps=1500,
)
trainer = tr.Trainer(
    gpt,
    args=targs,
    train_dataset=dsb["train"],
    tokenizer=ttokenizer,
    data_collator=tr.default_data_collator,  # tr.DataCollatorForLanguageModeling(tokenizer=ttokenizer,mlm=False)
)

<ipython-input-21-34047ea34211>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = tr.Trainer(


In [ ]:
trainer.train()

Теперь посмотрим, как работает генерация:

In [ ]:
#ttokenizer = tr.PreTrainedTokenizerFast(tokenizer_object=tokenizer)
res = gpt.generate(
    **ttokenizer("Пьер закашлялся и", return_tensors="pt").to("cuda"),
    max_new_tokens=150,
    do_sample=True
)
ttokenizer.decode(res[0])

'Пьер закашлялся и заверну обнимали обнимали приобрел ори подбородок Rapp молиться ваемый тонким скал нится главы простым чрезвычай lais ален прон известное папенька почт ходь смягчи почт стенки верст принца вольте прал эскадрон тости Шере скал роте обмунди неловкость тости we вытянув разоря ходь курган дования саживаясь министерства плодо деньгами Аграф блестел Собра склад фантазии швейца малень французское свадь ответственности щеголяя вре арфе другой старичку обвяза mante эман входи чное бую прон рассматриваем генералов двугри шительно ален пальцев подо диа Челове добросовестно тела нахожусь перекрестилась АЯ считаться эскадрон сердись поезд казенный жалоб чину Адъютан самые голодом жалоб подо нится эскадрон подбородок обвяза швейца швейца рассматриваем предназначено отступление озлобле высунулась таранта развлека внутренней Бона vou адвокаты отдачи нясь смех слушавший слушавший Аркадьича Степанович жденные шительно разоря ию подбородок входи вших приездом челю высунулась чка весели

Кажется, что сгенерированный текст пока ещё не слишком осмысленный. Но сравните его с первоначальным текстом, сгенерированным необученной нейросетью - в нём почти не было корректных грамматических конструкций. За примерно час обучения сеть уже стала неплохо понимать, какие слова хорошо сочетаются друг с другом, и в целом начала говорить более осмысленно. Помните, что трансформерная модель - сложная, и для обучения полноценной GPT-2 "с нуля" требуются сотни и тысячи GPU-часов.

> Прежде, чем переходить к следующим экспериментам, очистим память. Если вдруг на следующем этапе возникнет переполнение памяти GPU, может потребоваться перезапуск ядра ноутбука - выберите к меню Kernel -> Restart Kernel

In [ ]:
import gc

gpt = None
gc.collect()

68

## До-обучение GPT-2

За приемлемое время сложно достичь приемлемого качества обучения трансформера, поэтому обычно используют предобученные модели (поэтому в названии GPT и фигурирует слово *Pretrained*), которые уже научились "читать" на нужном языке, и их необходимо лишь немного "доучить" под требуемую предметную область или стиль. В этом случае процесс обучения модели почти не отличается от того, что мы делали ранее - с той лишь разницей, что необходимо использовать токенизатор, который использовался при обучении исходной модели.

Для начала, загрузим предобученную модель **ruGPT** и соответствующий токенизатор, и посмотрим, как эта модель умеет продолжать текст:

In [ ]:
tokenizer = tr.AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
gpt = tr.GPT2LMHeadModel.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
res = gpt.generate(
    **tokenizer("Мне нравится, что вы ", return_tensors="pt"),
    max_new_tokens=50,
    top_k=3,
    do_sample=True
)
tokenizer.decode(res[0])

'Мне нравится, что вы \nне \nзабываете  о\nнеобходимости \nпомнить \nо том, что \nвы \nсуществовали.\n\n- Я \nне \nзабываю  об\nэтом,'

На самом деле качество модели *очень сильно* зависит от количества параметров, и тот факт, что мы взяли модель **ruGPTsmall** сказывается на качестве текста. Но зато и процесс обучения будет существенно быстрее!

Поскольку мы теперь используем другой токенизатор, то нам нужно заново токенизировать датасет:

In [ ]:
dataset = datasets.load_dataset("text", data_files="dataset.txt")
ds = dataset.map(lambda x:
                 tokenizer(x["text"]), batched=True, remove_columns=["text"])
dsb = ds.map(group_texts, batched=True)

Сам по себе процесс запуска обучения и указания параметров ничем не отличается от обучения трансформерной модели "с нуля". Возможно, при до-обучении имеет смысл указывать чуть более низкий `learning_rate`.

In [ ]:
targs = tr.TrainingArguments(
    output_dir="gpt2-finetune",
    num_train_epochs=30,
    learning_rate=5e-5,
    warmup_steps=200,
    save_steps=1500,
)
trainer = tr.Trainer(
    gpt,
    args=targs,
    train_dataset=dsb["train"],
    tokenizer=tokenizer,
    data_collator=tr.default_data_collator,  # tr.DataCollatorForLanguageModeling(tokenizer=ttokenizer,mlm=False)
)
trainer.train()

<ipython-input-25-c8a8530a8982>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = tr.Trainer(


Step,Training Loss
500,3.288700
1000,3.047200
1500,2.899000


Step,Training Loss
500,3.288700
1000,3.047200
1500,2.899000
2000,2.802300


TrainOutput(global_step=2232, training_loss=2.983157715062514, metrics={'train_runtime': 3547.6508, 'train_samples_per_second': 5.031, 'train_steps_per_second': 0.629, 'total_flos': 6375933849600000.0, 'train_loss': 2.983157715062514, 'epoch': 8.0})

Смотрим на результат генерации после обучения:

In [ ]:
res = gpt.generate(
    **tokenizer("Толстой ", return_tensors="pt").to("cuda"),
    max_new_tokens=100,
    top_k=3,
    do_sample=True
)
tokenizer.decode(res[0])

"Идите на  войну! --  закричал  он,  указывая  на  французов,которые, не обращая  внимания на него, бежали за ним,  и,  не  видя  его,  бежали  за  другими  французами,которые, не  обращая  на него  никакого внимания, бежали за ним.--  Vive  c'est une  malade!  [43] --  закричал  он,  и,  не  обращая  нанего "

Кажется, что мы получили сильно более хороший результат!

## Параллелизация обучения

Надеюсь, вы убедились, что на DataSphere можно обучать достаточно мощные модели, однако время, затрачиваемое на обучение, всё ещё остаётся большим. Чтобы ускорить этот процесс, обычно используют параллельное обучение на нескольких GPU одновременно.

Самым распространённым вариантом параллелизма является параллелизм по данным (Data Parallel Training), в котором на каждый из обучающих GPU подаётся свой поток данных (т.е. своя часть исходного датасета). При этом на каждом обучающем шаге каждый GPU вычисляет свой градиент ошибки, которые затем усредняются и используются для синхронного обновления моделей на всех обучающих процессорах.

Различают два варианта обучения на нескольких GPU:
* **Data Parallel** - обычно используется, когда несколько GPU установлены на одном компьютере. В этом случае используется почти такой же код обучения на Python, как для однопроцессорного варианта, модель оборачивается в класс `torch.nn.DataParallel`, и минибатч распределяется по нескольким доступным на данном компьютере GPU.
* **Distributed Data Parallel** используется в более общем случае, когда есть кластер из компьютеров с GPU.

Подробнее про параллельное обучения можно почитать [в руководстве PyTorch](https://pytorch.org/docs/stable/distributed.html#distributed-basics).


## Заключение

Одна из целей данной работы заключалась в том, чтобы продемонстрировать, что обучение сложных языковых моделей с помощью современных библиотек является сравнительно простой задачей - но требующей значительных вычислительных ресурсов. Как только мы выходим за рамки вычислений, которые можно сделать за несколько часов на общедоступных инструментах типа Google Colab - у нас возникает потребность в облачных вычислительных ресурсах.

Yandex DataSphere обеспечивает легкий переход от локального Jupyter Notebook или публичного облака Google Colab / Kaggle к выделенной облачной инфраструктуре в Yandex Cloud. В DataSphere вы можете:

* легко настроить подключения к облачным хранилищам данных,
* взаимодействовать с другими участниками проекта
* использовать GitHub для контроля версий кода
* бережливо расходовать ресурсы благодаря режиму Serverless или возможности легкого переключения между виртуальными вычислителями

Для эффективной работы в DataSphere в ней необходимо немного привыкнуть, но когда этап привыкания пройдёт - вы сможете эффективно пользоваться этим инструментом и получать удовольствие от работы в нём!